In [22]:
"""A very simple MNIST classifier.
See extensive documentation at
https://www.tensorflow.org/get_started/mnist/beginners
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data



from keras.layers.core import Dense, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D

from keras import backend as K


K.image_data_format() 

'channels_last'

keras.layers.core.Dense(units, activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)                       
units：大于0的整数，代表该层的输出维度。                                                                                                
activation：激活函数，为预定义的激活函数名（参考激活函数），或逐元素（element-wise）的Theano函数。如果不指定该参数，将不会使用任何激活函数（即使用线性激活函数：a(x)=x）
use_bias: 布尔值，是否使用偏置项
kernel_initializer：权值初始化方法，为预定义初始化方法名的字符串，或用于初始化权重的初始化器。参考initializers                        
bias_initializer：偏置向量初始化方法，为预定义初始化方法名的字符串，或用于初始化偏置向量的初始化器。参考initializers                  
kernel_regularizer：施加在权重上的正则项，为Regularizer对象                                                                            
bias_regularizer：施加在偏置向量上的正则项，为Regularizer对象                                                                          
activity_regularizer：施加在输出上的正则项，为Regularizer对象                                                                          
kernel_constraints：施加在权重上的约束项，为Constraints对象                                                                            
bias_constraints：施加在偏置上的约束项，为Constraints对象                                                                              

In [23]:
#定义文件的位置
data_dir = '/input_data'
#读取并且下载
mnist = input_data.read_data_sets(data_dir, one_hot=True)

#定义一个占位符  代表x轴
x = tf.placeholder(tf.float32, [None, 784])#784 是图片28*28的列
#tf.float32  代表值的类型
#[None, 784]  行数未确定, 这里有784列

#定义一个占位符   代表预测的理想值
y_ = tf.placeholder(tf.float32, [None, 10])#10   是要出的理想结果
#tf.float32 代表值的类型
#[None, 10] 行数未确定,有10列   

#定义一个占位符   代表学习率
learning_rate = tf.placeholder(tf.float32)
#tf.float32  代表值的类型

#定义操作的名字
with tf.name_scope('reshape'):
  #定义张量积
  x_image = tf.reshape(x, [-1, 28, 28, 1])
  #-1  代表未知道输入的数目，这个可以变
  #28  代表图片的长为28
  #28  代表图片的宽为28
  #1   输出一个核

#定义卷积网络 
net = Conv2D(32, kernel_size=[5,5], strides=[1,1],activation='relu',padding='same',input_shape=[28,28,1])(x_image)
#32  指定的卷积核个数
#kernel_size=[5,5]  卷积核的大小
#strides=[1,1]  步长
#activation='relu'  激活的函数
#padding='same'  填充
#input_shape=[28,28,1]  读入的张量积

#定义最大池化
net = MaxPooling2D(pool_size=[4,4])(net)
#pool_size=[2,2]  定义池的大小

#定义卷积网络
net = Conv2D(64, kernel_size=[5,5], strides=[1,1],activation='relu',padding='same')(net)
#64  指定的卷积核个数
#kernel_size=[5,5]  卷积核的大小
#strides=[1,1]  步长
#activation='relu'  激活的函数
#padding='same'  填充

#定义最大池化
net = MaxPooling2D(pool_size=[3,3])(net)
#pool_size=[4,4]  定义池的大小

#定义卷积网络
net = Conv2D(128, kernel_size=[5,5], strides=[1,1],activation='relu',padding='same')(net)
#128  指定的卷积核个数
#kernel_size=[5,5]  卷积核的大小
#strides=[1,1]  步长
#activation='relu'  激活的函数
#padding='same'  填充

#定义最大池化
net = MaxPooling2D(pool_size=[2,2])(net)
#pool_size=[2,2]  定义池的大小



#压平池化
net = Flatten()(net)

#定义全连接
net = Dense(1000, activation='relu')(net)
#1000  输出的维度
#activation='relu'  激活函数

#定义全连接
net = Dense(10,activation='softmax')(net)
#10  输出的维度
#activation='softmax'  激活函数

Extracting /input_data\train-images-idx3-ubyte.gz
Extracting /input_data\train-labels-idx1-ubyte.gz
Extracting /input_data\t10k-images-idx3-ubyte.gz
Extracting /input_data\t10k-labels-idx1-ubyte.gz


In [24]:
from keras.objectives import categorical_crossentropy
#定义出交叉熵的平均值
cross_entropy = tf.reduce_mean(categorical_crossentropy(y_, net))
#categorical_crossentropy(y_, net)  算出交叉熵
#y_  理想的y值
#net  预测的值

In [25]:
#定义l2损失
l2_loss = tf.add_n( [tf.nn.l2_loss(w) for w in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)] )
#算出总的损失
total_loss = cross_entropy + 7e-5*l2_loss
#定义梯度下降的对象
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)
#learning_rate  学习率
#total_loss   总的损失

#定义一个session对象
sess = tf.Session()

In [26]:
#把kears设置都session中
K.set_session(sess)

In [27]:
#初始化所有的值
init_op = tf.global_variables_initializer()
#运行
sess.run(init_op)
# 进行训练
for step in range(5000):
  #拿出训练集的x轴和y周
  batch_xs, batch_ys = mnist.train.next_batch(100)
  #100  拿100个节点
  
  #定义学习率
  lr = 0.01
  
  #运行
  _, loss, l2_loss_value, total_loss_value = sess.run([train_step, cross_entropy, l2_loss, total_loss],feed_dict={x: batch_xs, y_: batch_ys, learning_rate:lr})
  #train_step  梯度下降的对象
  #cross_entropy  交叉熵
  #l2_loss  L2损失
  #total_loss总的损失
  #feed_dict={x: batch_xs, y_: batch_ys, learning_rate:lr}
  #x: batch_xs  x轴的训练集
  #y_: batch_ys y轴的结果集
  #learning_rate:lr  学习率
    
  if (step+1) % 100 == 0:
    print('step %d, entropy loss: %f, l2_loss: %f, total loss: %f' % (step+1, loss, l2_loss_value, total_loss_value))
    # Test trained model
    correct_prediction = tf.equal(tf.argmax(net, 1), tf.argmax(y_, 1))
    #tf.argmax 是一个非常有用的函数，它能给出某个tensor对象在某一维上的其数据最大值所在的索引值
    #tf.equal 来检测我们的预测是否真实标签匹配(索引位置一样表示匹配)
    
    #算出平均值
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    #tf.cast   [True, False, True, True] 会变成 [1,0,1,1] ，取平均值后得到 0.75
    
     #算出训练集得分
    print(sess.run(accuracy, feed_dict={x: batch_xs, y_: batch_ys}))
  if (step+1) % 1000 == 0:
    #算出测试集的得分
    print(sess.run(accuracy, feed_dict={x: mnist.test.images,y_: mnist.test.labels}))

step 100, entropy loss: 2.300767, l2_loss: 1228.569946, total loss: 2.386767
0.24
step 200, entropy loss: 2.292411, l2_loss: 1228.422607, total loss: 2.378401
0.26
step 300, entropy loss: 2.289448, l2_loss: 1228.292236, total loss: 2.375429
0.29
step 400, entropy loss: 2.280839, l2_loss: 1228.184448, total loss: 2.366812
0.31
step 500, entropy loss: 2.272893, l2_loss: 1228.105835, total loss: 2.358860
0.41
step 600, entropy loss: 2.268124, l2_loss: 1228.073730, total loss: 2.354089
0.42
step 700, entropy loss: 2.245479, l2_loss: 1228.122314, total loss: 2.331447
0.38
step 800, entropy loss: 2.190274, l2_loss: 1228.311035, total loss: 2.276256
0.5
step 900, entropy loss: 2.093399, l2_loss: 1228.796875, total loss: 2.179415
0.5
step 1000, entropy loss: 1.785847, l2_loss: 1229.989990, total loss: 1.871947
0.55
0.5651
step 1100, entropy loss: 1.404231, l2_loss: 1232.016235, total loss: 1.490472
0.7
step 1200, entropy loss: 1.074254, l2_loss: 1234.106201, total loss: 1.160641
0.63
step 1300